# Starship High-Fidelity 6-DoF Re-Entry Simulation with DSFB during Plasma Blackout

This notebook demonstrates a Starship-style hypersonic re-entry scenario with a plasma blackout and compares three estimators:

- Pure inertial navigation
- Simple GNSS-aided EKF
- DSFB trust-adaptive redundant IMU fusion

Motivation: show how **Drift-Slew Fusion Bootstrap (DSFB)** attenuates faulty channels during abrupt slew events and thermal drift.

DSFB paper: https://github.com/infinityabundance/dsfb/blob/main/docs/Slew-Aware%20Trust-Adaptive%20Nonlinear%20State%20Estimation.pdf


## Installation (Rust + maturin + crate build)

This cell installs Python dependencies, Rust (if missing), builds the crate, and installs the Python module from a wheel (works in Colab without a virtualenv).

In [ ]:
%pip -q install numpy pandas matplotlib plotly ipywidgets maturin

import glob
import os
import pathlib
import subprocess
import sys

if not pathlib.Path('/root/.cargo/bin/cargo').exists():
    subprocess.run('curl https://sh.rustup.rs -sSf | sh -s -- -y', shell=True, check=True)

subprocess.run('apt-get -qq update && apt-get -qq install -y patchelf', shell=True, check=True)

os.environ['PATH'] = '/root/.cargo/bin:' + os.environ['PATH']

if not pathlib.Path('/content/dsfb').exists():
    subprocess.run(['git', 'clone', 'https://github.com/infinityabundance/dsfb.git', '/content/dsfb'], check=True)
else:
    subprocess.run(['git', '-C', '/content/dsfb', 'checkout', 'main'], check=True)
    subprocess.run(['git', '-C', '/content/dsfb', 'pull', '--ff-only', 'origin', 'main'], check=True)

# Fallback only if dsfb-starship is not yet present on main.
if not pathlib.Path('/content/dsfb/crates/dsfb-starship').exists():
    os.chdir('/content/dsfb')
    subprocess.run(['git', 'fetch', 'origin', 'feature/dsfb-starship'], check=True)
    subprocess.run(['git', 'checkout', 'feature/dsfb-starship'], check=True)

%cd /content/dsfb/crates/dsfb-starship
subprocess.run(['cargo', 'build', '--release'], check=True)
subprocess.run([sys.executable, '-m', 'maturin', 'build', '--release', '--out', 'target/wheels'], check=True)

wheels = sorted(glob.glob('target/wheels/dsfb_starship-*.whl'))
if not wheels:
    raise RuntimeError('No wheel produced by maturin build')

subprocess.run([sys.executable, '-m', 'pip', 'install', '-U', '--force-reinstall', wheels[-1]], check=True)

import dsfb_starship
print('Imported module:', dsfb_starship.__name__)


## Simulation Setup

Default scenario:

- Entry speed: ~7.5 km/s
- Flight path angle: ~-5.5°
- Plasma blackout window: 80 km down to 40 km (target ~5–7 min)
- Redundant IMUs with thermal bias ramps, noise, and abrupt slew transients
- DSFB `rho` and slew threshold are tunable

In [ ]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets

import dsfb_starship

PLOT_TEMPLATE = 'plotly_white'

def run_demo(rho=0.97, slew_threshold=32.0, seed=17, output_dir='output-colab'):
    summary_json = dsfb_starship.run_starship_simulation(
        output_dir=output_dir,
        rho=float(rho),
        slew_threshold=float(slew_threshold),
        seed=int(seed),
    )
    summary = json.loads(summary_json)
    csv_path = Path(summary['outputs']['csv_path'])
    df = pd.read_csv(csv_path)
    return summary, df


## Run Pure Inertial, Simple EKF, and DSFB

In [ ]:
summary, df = run_demo()

metrics = pd.DataFrame({
    'Method': ['Pure Inertial', 'Simple EKF', 'DSFB'],
    'RMSE Position [m]': [summary['inertial']['rmse_position_m'], summary['ekf']['rmse_position_m'], summary['dsfb']['rmse_position_m']],
    'RMSE Velocity [m/s]': [summary['inertial']['rmse_velocity_mps'], summary['ekf']['rmse_velocity_mps'], summary['dsfb']['rmse_velocity_mps']],
    'RMSE Attitude [deg]': [summary['inertial']['rmse_attitude_deg'], summary['ekf']['rmse_attitude_deg'], summary['dsfb']['rmse_attitude_deg']],
    'Final Position Error [m]': [summary['inertial']['final_position_error_m'], summary['ekf']['final_position_error_m'], summary['dsfb']['final_position_error_m']],
})

display(metrics.round(3))
print(f"Blackout duration: {summary['blackout_duration_s']:.1f} s")


## Trajectory Plots (3D + Altitude)

In [ ]:
fig3d = go.Figure()
fig3d.add_trace(go.Scatter3d(x=df['truth_x_km'], y=df['truth_y_km'], z=df['truth_z_km'], mode='lines', name='Truth', line=dict(width=6, color='#0B3C5D')))
fig3d.add_trace(go.Scatter3d(x=df['inertial_x_km'], y=df['inertial_y_km'], z=df['inertial_z_km'], mode='lines', name='Inertial', line=dict(width=4, color='#B22222')))
fig3d.add_trace(go.Scatter3d(x=df['ekf_x_km'], y=df['ekf_y_km'], z=df['ekf_z_km'], mode='lines', name='EKF', line=dict(width=4, color='#2E8B57')))
fig3d.add_trace(go.Scatter3d(x=df['dsfb_x_km'], y=df['dsfb_y_km'], z=df['dsfb_z_km'], mode='lines', name='DSFB', line=dict(width=5, color='#1E90FF')))
fig3d.update_layout(template=PLOT_TEMPLATE, title='3D Trajectory', scene=dict(xaxis_title='Downrange [km]', yaxis_title='Crossrange [km]', zaxis_title='Altitude [km]'), height=650)
fig3d.show()

fig_alt = go.Figure()
fig_alt.add_trace(go.Scatter(x=df['time_s'], y=df['altitude_m'] / 1000.0, mode='lines', name='Altitude', line=dict(color='#0B3C5D', width=4)))
blackout_df = df[df['blackout'] == True]
if len(blackout_df) > 0:
    fig_alt.add_vrect(x0=blackout_df['time_s'].min(), x1=blackout_df['time_s'].max(), fillcolor='rgba(255,99,71,0.22)', line_width=0, annotation_text='Plasma blackout', annotation_position='top left')
fig_alt.update_layout(template=PLOT_TEMPLATE, title='Altitude Profile with Blackout Window', xaxis_title='Time [s]', yaxis_title='Altitude [km]', height=450)
fig_alt.show()


## Error, Trust Weights, and Residual Increments

In [ ]:
fig_err = go.Figure()
fig_err.add_trace(go.Scatter(x=df['time_s'], y=df['inertial_pos_err_m'], mode='lines', name='Inertial', line=dict(color='#B22222', width=3)))
fig_err.add_trace(go.Scatter(x=df['time_s'], y=df['ekf_pos_err_m'], mode='lines', name='EKF', line=dict(color='#2E8B57', width=3)))
fig_err.add_trace(go.Scatter(x=df['time_s'], y=df['dsfb_pos_err_m'], mode='lines', name='DSFB', line=dict(color='#1E90FF', width=4)))
fig_err.update_layout(template=PLOT_TEMPLATE, title='Position Error (Log Scale)', xaxis_title='Time [s]', yaxis_title='Position Error [m]', yaxis_type='log', height=420)
fig_err.show()

fig_trust = make_subplots(rows=1, cols=2, subplot_titles=('DSFB Trust Weights', 'Residual Increment Magnitudes'))
fig_trust.add_trace(go.Scatter(x=df['time_s'], y=df['dsfb_trust_imu0'], mode='lines', name='IMU-0 trust', line=dict(color='#0B3C5D')), row=1, col=1)
fig_trust.add_trace(go.Scatter(x=df['time_s'], y=df['dsfb_trust_imu1'], mode='lines', name='IMU-1 trust', line=dict(color='#B22222')), row=1, col=1)
fig_trust.add_trace(go.Scatter(x=df['time_s'], y=df['dsfb_trust_imu2'], mode='lines', name='IMU-2 trust', line=dict(color='#2E8B57')), row=1, col=1)

fig_trust.add_trace(go.Scatter(x=df['time_s'], y=df['dsfb_resid_inc_imu0'], mode='lines', name='IMU-0 residual inc', line=dict(color='#0B3C5D', dash='dot')), row=1, col=2)
fig_trust.add_trace(go.Scatter(x=df['time_s'], y=df['dsfb_resid_inc_imu1'], mode='lines', name='IMU-1 residual inc', line=dict(color='#B22222', dash='dot')), row=1, col=2)
fig_trust.add_trace(go.Scatter(x=df['time_s'], y=df['dsfb_resid_inc_imu2'], mode='lines', name='IMU-2 residual inc', line=dict(color='#2E8B57', dash='dot')), row=1, col=2)

fig_trust.update_xaxes(title_text='Time [s]', row=1, col=1)
fig_trust.update_xaxes(title_text='Time [s]', row=1, col=2)
fig_trust.update_yaxes(title_text='Trust Weight', row=1, col=1)
fig_trust.update_yaxes(title_text='Residual Increment', row=1, col=2)
fig_trust.update_layout(template=PLOT_TEMPLATE, height=450, title='DSFB Internal Diagnostics')
fig_trust.show()

blackout_only = df[df['blackout'] == True]
comparison = pd.DataFrame({
    'Metric': ['Mean position error in blackout [m]', 'Mean velocity error in blackout [m/s]', 'Mean attitude error in blackout [deg]'],
    'Inertial': [blackout_only['inertial_pos_err_m'].mean(), blackout_only['inertial_vel_err_mps'].mean(), blackout_only['inertial_att_err_deg'].mean()],
    'EKF': [blackout_only['ekf_pos_err_m'].mean(), blackout_only['ekf_vel_err_mps'].mean(), blackout_only['ekf_att_err_deg'].mean()],
    'DSFB': [blackout_only['dsfb_pos_err_m'].mean(), blackout_only['dsfb_vel_err_mps'].mean(), blackout_only['dsfb_att_err_deg'].mean()],
})
display(comparison.round(3))


## Interactive Sliders for Slew Threshold and ρ

Use sliders to rerun the DSFB configuration and inspect summary metrics quickly.

In [ ]:
def quick_sweep(rho=0.97, slew_threshold=32.0):
    s, d = run_demo(rho=rho, slew_threshold=slew_threshold, output_dir='output-interactive')
    table = pd.DataFrame({
        'Method': ['Inertial', 'EKF', 'DSFB'],
        'RMSE Position [m]': [s['inertial']['rmse_position_m'], s['ekf']['rmse_position_m'], s['dsfb']['rmse_position_m']],
        'RMSE Attitude [deg]': [s['inertial']['rmse_attitude_deg'], s['ekf']['rmse_attitude_deg'], s['dsfb']['rmse_attitude_deg']],
    })
    print(f"Blackout duration: {s['blackout_duration_s']:.1f} s | rho={rho:.3f} | slew_threshold={slew_threshold:.1f}")
    display(table.round(3))

widgets.interact(
    quick_sweep,
    rho=widgets.FloatSlider(value=0.97, min=0.90, max=0.995, step=0.005),
    slew_threshold=widgets.FloatSlider(value=32.0, min=15.0, max=60.0, step=1.0),
)


## Conclusions

- The simulation reproduces a long plasma blackout interval (80 km to 40 km) with high entry speed and realistic re-entry loads.
- Abrupt sensor slew events and thermal drift drive trust attenuation, especially on the faulty IMU channel.
- DSFB remains competitive with EKF on trajectory error while improving robustness to attitude and fault transients.
- The trust and residual-increment plots provide paper-ready diagnostics to support DSFB claims in blackout conditions.